# Distilling Knowledge in Multiple Students  using Generative models

In [1]:
# !nvidia-smi

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils as np_utils

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Teacher's Training

In [2]:
#Loading and splitting the dataset into train, validation and test
nb_classes = 10

(X_Train, y_Train), (X_test, y_test) = mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

X_train = X_train.reshape(48000, 28, 28, 1)
X_val = X_val.reshape(12000, 28, 28, 1)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255


#Creating a teacher network
input_shape = (28, 28, 1) # Input shape of each image

teacher = Sequential()
teacher.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
teacher.add(Conv2D(64, (3, 3), activation='relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))

teacher.add(Dropout(0.25)) # For reguralization

teacher.add(Flatten())
teacher.add(Dense(256, activation='relu'))
teacher.add(Dense(256, activation='relu', name="dense_1"))

teacher.add(Dropout(0.5)) # For reguralization

teacher.add(Dense(nb_classes, name = 'dense_2'))
teacher.add(Activation('softmax')) # Note that we add a normal softmax layer to begin with

teacher.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

print(teacher.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2359552   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6

In [ ]:
# Train the teacher model as usual
epochs = 0
batch_size = 256
teacher.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, Y_val))


### Evaluation 

In [3]:
teacher.load_weights("Teacher_MNIST_98.h5")

In [4]:
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [5]:
X_test = X_test.reshape(10000, 28, 28, 1)

X_test = X_test.astype('float32')
# Normalize the values
X_test /= 255

In [6]:
teacher.evaluate(X_test,Y_test)

313/313 [==============================] - 2s 3ms/step - loss: 0.0484 - accuracy: 0.9874


[0.03833923488855362, 0.9894999861717224]

### Dense Vector split

In [7]:
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)

# 8 Students case
# ---------------------------------------------
s1Train=train_dense[:,:32]
s2Train=train_dense[:,32:64]
s3Train=train_dense[:,64:96]
s4Train=train_dense[:,96:128]
s5Train=train_dense[:,128:160]
s6Train=train_dense[:,160:192]
s7Train=train_dense[:,192:224]
s8Train=train_dense[:,224:]

## GANs' Training

In [8]:
# import np.random import random
BATCH_SIZE=32
def smooth_real_labels(y):
    return y - 0.3+(np.random.random(y.shape)*0.5)
def smooth_fake_labels(y):
    return y + (0.3 * np.random.random(y.shape))
def build_gan(gen,disc): 
    disc.trainable = False
    input= Input(shape=input_shape)
    output = gen(input)
    output2= disc(output)
    gan=Model(input,output2)

    gan.compile(Adam(lr=0.0002),loss=['binary_crossentropy','mse'],metrics=['accuracy'])

    return gan

### Discriminator

In [9]:
def build_sdiscriminator():
    
    input2 = Input(shape=(32,),name='input')
    inp=Dense(64)(input2)

    leaky_relu = LeakyReLU(alpha=0.2)(inp)
    
    conv3 = Dense(128)(leaky_relu)
    b_n = BatchNormalization()(conv3)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(256)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(512)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(1024)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)

    dense = Dense(1,activation='sigmoid',name='dense')(leaky_relu)

    output2=Dense(32)(leaky_relu)

    
    disc = Model(input2,[dense,output2])          
    disc.compile(optd,loss=['binary_crossentropy','mse'],metrics=['accuracy'])


    return disc

In [10]:
optd = Adam(lr=0.0002)
opt = Adam(lr=0.0002)

In [11]:
# d1= build_sdiscriminator()
# d1.summary()

### Generator

In [14]:
def build_sgenerator(name):

    student1 = Sequential()
    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 1),kernel_initializer='normal', name=name))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',kernel_initializer='normal'))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Flatten())
    student1.add(Dense(8, activation='relu'))
    student1.add(Dropout(0.2))
    student1.add(Dense(32,name='req'+name))

    student1.compile(opt,loss='mean_squared_error',metrics=['accuracy'])
    
    return student1

build_sgenerator('').summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 24, 24, 16)        2320      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 10, 10, 16)        2320      
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 8, 8, 16)          2320      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 256)              

In [15]:
def training(generator,discriminator,gan,features,epo=20):
    # Setup Models here
    BATCH_SIZE = 128
    discriminator.trainable = True
    total_size = X_train.shape[0]
    indices = np.arange(0,total_size ,BATCH_SIZE)
    all_disc_loss = []
    all_gen_loss = []
    all_class_loss=[]
    if total_size % BATCH_SIZE:
        indices = indices[:-1]
    for e in range(epo):
        
        progress_bar = Progbar(target=len(indices))
        np.random.shuffle(indices)
        epoch_gen_loss = []
        epoch_disc_loss = []
        epoch_class_loss= []
        for i,index in enumerate(indices):
        
            # Split
            inputs=X_train[index:index+BATCH_SIZE]
            sXtrain = features[index:index+BATCH_SIZE]

            y_real = np.ones((BATCH_SIZE,1))
            y_fake = np.zeros((BATCH_SIZE,1))

            #Generator Training
            fake_images = generator.predict_on_batch(inputs)

            #Disrciminator Training
            disc_real_loss1,_,disc_real_loss2,_,_= discriminator.train_on_batch(sXtrain,[y_real,sXtrain])
            disc_fake_loss1,_,disc_fake_loss2,_,_= discriminator.train_on_batch(fake_images,[y_fake,sXtrain])

            #Gans Training
            discriminator.trainable = False
            gan_loss,_,gan_loss2,_,_ = gan.train_on_batch(inputs, [y_real,sXtrain])

            discriminator.trainable = True

            disc_loss = (disc_fake_loss1 + disc_real_loss1)/2
            epoch_disc_loss.append(disc_loss)
            progress_bar.update(i+1)

            epoch_gen_loss.append((gan_loss))

        avg_epoch_disc_loss = np.array(epoch_disc_loss).mean()
        avg_epoch_gen_loss = np.array(epoch_gen_loss).mean()
        all_disc_loss.append(avg_epoch_disc_loss)
        all_gen_loss.append(avg_epoch_gen_loss)
        print("Epoch: %d | Discriminator Loss: %f | Generator Loss: %f | " % (e+1,avg_epoch_disc_loss,avg_epoch_gen_loss))

    return generator

You can initialize multiple instances here 

In [16]:
discriminator1 = build_sdiscriminator()
discriminator2 = build_sdiscriminator()
discriminator3 = build_sdiscriminator()
discriminator4 = build_sdiscriminator()
discriminator5 = build_sdiscriminator()
discriminator6 = build_sdiscriminator()
discriminator7 = build_sdiscriminator()
discriminator8 = build_sdiscriminator()


In [17]:
s1=build_sgenerator("s1")
s2=build_sgenerator('s2')
s3=build_sgenerator("s3")
s4=build_sgenerator('s4')
s5=build_sgenerator("s5")
s6=build_sgenerator('s6')
s7=build_sgenerator("s7")
s8=build_sgenerator('s8')


In [18]:
gan1 = build_gan(s1,discriminator1)
gan2 = build_gan(s2,discriminator2)
gan3 = build_gan(s3,discriminator3)
gan4 = build_gan(s4,discriminator4)
gan5 = build_gan(s5,discriminator5)
gan6 = build_gan(s6,discriminator6)
gan7 = build_gan(s7,discriminator7)
gan8 = build_gan(s8,discriminator8)

In [19]:
s1 = training(s1,discriminator1,gan1,s1Train,epo=40)
s2 = training(s2,discriminator2,gan2,s2Train,epo=35)
s3 = training(s3,discriminator3,gan3,s3Train,epo=40)
s4 = training(s4,discriminator4,gan4,s4Train,epo=35)

375/375 [==============================] - 14s 35ms/step
Epoch: 1 | Discriminator Loss: 0.610805 | Generator Loss: 1.859144 | 
375/375 [==============================] - 14s 36ms/step
Epoch: 2 | Discriminator Loss: 0.258086 | Generator Loss: 1.366812 | 
375/375 [==============================] - 14s 36ms/step
Epoch: 3 | Discriminator Loss: 0.202305 | Generator Loss: 1.261209 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 4 | Discriminator Loss: 0.181961 | Generator Loss: 1.220852 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 5 | Discriminator Loss: 0.169446 | Generator Loss: 1.150024 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 6 | Discriminator Loss: 0.162774 | Generator Loss: 1.079804 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 7 | Discriminator Loss: 0.159599 | Generator Loss: 1.012977 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 8 | Discriminator Loss: 0.156496 | Generator Lo

375/375 [==============================] - 13s 35ms/step
Epoch: 25 | Discriminator Loss: 0.111280 | Generator Loss: 0.666995 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 26 | Discriminator Loss: 0.109028 | Generator Loss: 0.659777 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 27 | Discriminator Loss: 0.107637 | Generator Loss: 0.658450 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 28 | Discriminator Loss: 0.108057 | Generator Loss: 0.653270 | 
375/375 [==============================] - 12s 32ms/step
Epoch: 29 | Discriminator Loss: 0.107762 | Generator Loss: 0.645326 | 
375/375 [==============================] - 12s 32ms/step
Epoch: 30 | Discriminator Loss: 0.108339 | Generator Loss: 0.642249 | 
375/375 [==============================] - 12s 31ms/step
Epoch: 31 | Discriminator Loss: 0.106973 | Generator Loss: 0.626940 | 
375/375 [==============================] - 12s 33ms/step
Epoch: 32 | Discriminator Loss: 0.106111 | Gene

375/375 [==============================] - 13s 35ms/step
Epoch: 14 | Discriminator Loss: 0.149544 | Generator Loss: 0.932659 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 15 | Discriminator Loss: 0.152267 | Generator Loss: 0.976963 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 16 | Discriminator Loss: 0.153405 | Generator Loss: 0.938478 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 17 | Discriminator Loss: 0.171029 | Generator Loss: 0.987870 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 18 | Discriminator Loss: 0.837781 | Generator Loss: 2.998919 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 19 | Discriminator Loss: 0.768313 | Generator Loss: 2.834702 | 
375/375 [==============================] - 14s 37ms/step
Epoch: 20 | Discriminator Loss: 0.766245 | Generator Loss: 2.553581 | 
375/375 [==============================] - 14s 36ms/step
Epoch: 21 | Discriminator Loss: 0.765208 | Gene

In [20]:
s5 = training(s5,discriminator5,gan5,s5Train,epo=40)
s6 = training(s6,discriminator6,gan6,s6Train,epo=35)
s7 = training(s7,discriminator7,gan7,s7Train,epo=40)
s8 = training(s8,discriminator8,gan8,s8Train,epo=35)

375/375 [==============================] - 14s 35ms/step
Epoch: 1 | Discriminator Loss: 0.602180 | Generator Loss: 2.024832 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 2 | Discriminator Loss: 0.316920 | Generator Loss: 1.462762 | 
375/375 [==============================] - 14s 38ms/step
Epoch: 3 | Discriminator Loss: 0.256590 | Generator Loss: 1.336074 | 
375/375 [==============================] - 14s 37ms/step
Epoch: 4 | Discriminator Loss: 0.210634 | Generator Loss: 1.210713 | 
375/375 [==============================] - 14s 38ms/step
Epoch: 5 | Discriminator Loss: 0.190840 | Generator Loss: 1.096297 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 6 | Discriminator Loss: 0.181825 | Generator Loss: 1.021782 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 7 | Discriminator Loss: 0.177091 | Generator Loss: 0.991264 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 8 | Discriminator Loss: 0.171314 | Generator Lo

375/375 [==============================] - 13s 35ms/step
Epoch: 25 | Discriminator Loss: 0.478025 | Generator Loss: 1.272705 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 26 | Discriminator Loss: 0.471157 | Generator Loss: 1.252849 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 27 | Discriminator Loss: 0.459086 | Generator Loss: 1.304870 | 
375/375 [==============================] - 14s 37ms/step
Epoch: 28 | Discriminator Loss: 0.454498 | Generator Loss: 1.332359 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 29 | Discriminator Loss: 0.498249 | Generator Loss: 1.374321 | 
375/375 [==============================] - 14s 37ms/step
Epoch: 30 | Discriminator Loss: 0.473220 | Generator Loss: 1.322039 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 31 | Discriminator Loss: 0.455974 | Generator Loss: 1.359948 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 32 | Discriminator Loss: 0.430157 | Gene

375/375 [==============================] - 13s 34ms/step
Epoch: 14 | Discriminator Loss: 0.187353 | Generator Loss: 1.008779 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 15 | Discriminator Loss: 0.179574 | Generator Loss: 0.945480 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 16 | Discriminator Loss: 0.178816 | Generator Loss: 0.934864 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 17 | Discriminator Loss: 0.174315 | Generator Loss: 0.953885 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 18 | Discriminator Loss: 0.170118 | Generator Loss: 0.953975 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 19 | Discriminator Loss: 0.171914 | Generator Loss: 0.959371 | 
375/375 [==============================] - 14s 37ms/step
Epoch: 20 | Discriminator Loss: 0.161668 | Generator Loss: 0.943163 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 21 | Discriminator Loss: 0.159665 | Gene

## **8 Students**

In [21]:
o1=s1.get_layer("reqs1").output
o2=s2.get_layer("reqs2").output
o3=s3.get_layer("reqs3").output
o4=s4.get_layer("reqs4").output

o5=s5.get_layer("reqs5").output
o6=s6.get_layer("reqs6").output
o7=s7.get_layer("reqs7").output
o8=s8.get_layer("reqs8").output

output=tensorflow.keras.layers.concatenate([o1,o2,o3,o4, o5,o6,o7,o8])
print (output.shape)
output=Activation('relu')(output)
output2=Dropout(0.5)(output) # For reguralization
output3=Dense(10,activation="softmax", name="d1")(output2)
mm8=Model([s1.get_layer("s1").input,s2.get_layer("s2").input,s3.get_layer("s3").input,s4.get_layer("s4").input,
                  s5.get_layer("s5").input, s6.get_layer("s6").input,s7.get_layer("s7").input,s8.get_layer("s8").input],output3)
my_weights=teacher.get_layer('dense_2').get_weights()

(None, 256)


In [22]:
# i=0
for l in mm8.layers[:len(mm8.layers)-1]:
    l.trainable=False

mm8.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [ ]:
batch_size = 128
mm8_history=mm8.fit([X_train,X_train,X_train,X_train,X_train,X_train,X_train,X_train], Y_train,
          batch_size=batch_size,
          epochs=100,
          verbose=1, validation_data=([X_val, X_val, X_val, X_val,X_val, X_val, X_val, X_val], Y_val))

Epoch 1/100
375/375 [==============================] - 7s 16ms/step - loss: 1.4075 - accuracy: 0.6342 - val_loss: 0.0824 - val_accuracy: 0.9758
Epoch 2/100
375/375 [==============================] - 5s 13ms/step - loss: 0.1955 - accuracy: 0.9433 - val_loss: 0.0768 - val_accuracy: 0.9778
Epoch 3/100
375/375 [==============================] - 5s 12ms/step - loss: 0.1589 - accuracy: 0.9552 - val_loss: 0.0757 - val_accuracy: 0.9780
Epoch 4/100
375/375 [==============================] - 5s 14ms/step - loss: 0.1487 - accuracy: 0.9565 - val_loss: 0.0738 - val_accuracy: 0.9781
Epoch 5/100
375/375 [==============================] - 5s 13ms/step - loss: 0.1370 - accuracy: 0.9615 - val_loss: 0.0716 - val_accuracy: 0.9787
Epoch 6/100
375/375 [==============================] - 5s 13ms/step - loss: 0.1328 - accuracy: 0.9641 - val_loss: 0.0710 - val_accuracy: 0.9789
Epoch 7/100
375/375 [==============================] - 5s 14ms/step - loss: 0.1352 - accuracy: 0.9620 - val_loss: 0.0720 - val_accuracy:

In [ ]:
loss, acc = mm8.evaluate([X_test,X_test,X_test,X_test,X_test,X_test,X_test,X_test], Y_test, verbose=1)
loss, acc

In [ ]:
mm8.summary()